In [1]:
import pandas as pd
import json
import numpy as np

In [57]:
megye={'Fehér':'ALBA', 'Arad':'ARAD', 'Bukarest':'B', 'Bákó':'BACAU', 'Bihar':'BIHOR', 'Beszterce-Naszód':'BISTRITA-NASAUD',
       'Brassó':'BRASOV', 'Kolozs':'CLUJ', 'Kovászna':'COVASNA', 'Krassó-Szörény':'CARAS-SEVERIN', 'Hunyad':'HUNEDOARA',
       'Hargita':'HARGHITA', 'Máramaros':'MARAMURES', 'Maros':'MURES', 'Szeben':'SIBIU', 'Szatmár':'SATU MARE', 'Szilágy':'SALAJ',
       'Temes':'TIMIS'}

In [58]:
ro={'Ă':'A','Ş':'S','Â':'A','Ș':'S','Ț':'T','Â':'A','Î':'I','Ă':'A','Ţ':'T','-':' ','SC.GEN.':'','I VIII':''}
def roman(s):
    return replacer(s,ro)

Load processed geocoded db

In [277]:
data=pd.read_excel('data/clean/erdely6.xlsx').drop('Unnamed: 0',axis=1)

In [278]:
data['guess_scores']=abs(data['guess_scores']).replace(0,50)

In [279]:
data.index=data['Denumire'].astype(str)+' '+data['Localitate'].astype(str)+', '+data['Localitate superioară'].astype(str)+', '+\
     data['Stradă'].astype(str)+' nr. '+data['Număr'].astype(str)+', '+data['Cod poștal'].astype(str).str[:-2]+', '+\
     data['Judet'].astype(str)+', ROMANIA'

In [280]:
geo=pd.read_excel('data/clean/geo.xlsx').drop('Unnamed: 0',axis=1).set_index('index')
geo['telepules_g']=geo['telepules']
geo=geo.drop('telepules',axis=1)

In [281]:
data=data.join(geo)

In [282]:
hun_city={i:i for i in np.sort(list(data['varos'].unique()))}

In [283]:
open('data/geo/hun_city.json','w',encoding='utf8').write(json.dumps(hun_city,ensure_ascii=False))

28264

In [284]:
pd.DataFrame(data['varos'].unique()).to_excel('data/geo/geo.xlsx')
# pd.DataFrame(data['varos'].unique()).to_excel('data/geo/geo_manual.xlsx')

Manually edit and fix, then load back

In [285]:
geom=list(pd.read_excel('data/geo/geo_manual.xlsx').drop('Unnamed: 0',axis=1)[0].unique())

In [286]:
geom=data[['telepules','varos']].set_index('varos').loc[geom].reset_index().set_index('telepules')
geom.columns=['varos_geo']

In [287]:
#can't join, no judet

Geocode from Szekelydata DB

In [288]:
hun=json.loads(open('data/geo/hun2.json','r').read())

In [289]:
hdf=pd.DataFrame(hun).stack().reset_index().set_index('level_1').join(pd.DataFrame(megye,index=['level_1']).T.reset_index().reset_index().set_index('level_1').drop('level_0',axis=1))
hdf.columns=['telepules','telepules_hun','Megye']

In [290]:
hdf.index=hdf['Megye']+'+'+hdf['telepules']
data.index=data['Megye']+'+'+data['telepules']

In [291]:
data=data.join(hdf['telepules_hun'])

In [292]:
data['telepules_hun']=data[['varos','telepules_hun']].T.ffill().T['telepules_hun']

In [314]:
gata=data[['ID','Év','Megye', 'telepules','telepules_hun','guessed_names2', 'guess_scores','Név','Típus', 'Profil',
       'Óvodás csoportok összesen',
       'Óvodások összesen', 'Kiscsoportok száma', 'Kiscsoportosok',
       'Középcsoportok száma', 'Középcsoportosok', 'Nagycsoportok száma',
       'Nagycsoportosok', 'Vegyes csoportok száma', 'Vegyescsoportosok',
       'Tanítók összesen', 'Képzett tanítók', 'Képzetlen tanítók',
       'Elemi osztályok összesen', 'Elemisek összesen',
       'Előkészítő osztályok száma', 'Előkészítő osztályosok',
       '1. osztályok száma', '1. osztályosok', '2. osztályok száma',
       '2. osztályosok', '3. osztályok száma', '3. osztályosok',
       '4. osztályok száma', '4. osztályosok', 'Általános osztályok összesen',
       'Általánososok összesen', '5. osztályok száma', '5. osztályosok',
       '6. osztályok száma', '6. osztályosok', '7. osztályok száma',
       '7. osztályosok', '8. osztályok száma', '8. osztályosok',
       'Középiskolai osztályok összesen', 'Középiskolások összesen',
       '9. osztályok száma', '9. osztályosok', '10. osztályok száma',
       '10. osztályosok', '11. osztályok száma', '11. osztályosok',
       '12. osztályok száma', '12. osztályosok', '13. osztályok száma',
       '13. osztályosok', '14. osztályok száma', '14. osztályosok','Továbbtanulás', 'Iskolabusz', 
        'Cod SIIIR', 'Cod SIRUES', 'Denumire scurtă', 'Denumire', 'Localitate',
       'Localitate superioară', 'Stradă', 'Număr', 'Cod poștal', 'Statut',
       'Tip unitate', 'Unitate PJ', 'Mod funcționare', 'Formă de finanțare',
       'Formă de proprietate', 'Cod fiscal', 'Judet', 'Data modificării',
       'Data acreditării', 'Data intrării în vigoare', 'Data închiderii',
       'Telefon', 'Fax', 'Adresa email', 'nev',  'telepules_g','varos','cim', 'koordinata', 'telefon', 'web', 'maps', 'kep', 
         
       ]]

In [320]:
gata.columns=['ID','Év','Megye', 'Település (eredeti)','Település (magyar VÁZLAT)','Név (normalizált)', 'Adatok megbízhatósága',
              'Név (eredeti)','Típus (VÁZLAT)', 'Profil (VÁZLAT)',
       'Óvodás csoportok összesen',
       'Óvodások összesen', 'Kiscsoportok száma', 'Kiscsoportosok',
       'Középcsoportok száma', 'Középcsoportosok', 'Nagycsoportok száma',
       'Nagycsoportosok', 'Vegyes csoportok száma', 'Vegyescsoportosok',
       'Tanítók összesen', 'Képzett tanítók', 'Képzetlen tanítók',
       'Elemi osztályok összesen', 'Elemisek összesen',
       'Előkészítő osztályok száma', 'Előkészítő osztályosok',
       '1. osztályok száma', '1. osztályosok', '2. osztályok száma',
       '2. osztályosok', '3. osztályok száma', '3. osztályosok',
       '4. osztályok száma', '4. osztályosok', 'Általános osztályok összesen',
       'Általánososok összesen', '5. osztályok száma', '5. osztályosok',
       '6. osztályok száma', '6. osztályosok', '7. osztályok száma',
       '7. osztályosok', '8. osztályok száma', '8. osztályosok',
       'Középiskolai osztályok összesen', 'Középiskolások összesen',
       '9. osztályok száma', '9. osztályosok', '10. osztályok száma',
       '10. osztályosok', '11. osztályok száma', '11. osztályosok',
       '12. osztályok száma', '12. osztályosok', '13. osztályok száma',
       '13. osztályosok', '14. osztályok száma', '14. osztályosok','Továbbtanulás (VÁZLAT)', 'Iskolabusz (VÁZLAT)', 
        'RSH SIIIR kód', 'RSH SIRUES kód', 'RSH Rövid név', 'RSH Név', 'RSH Település',
       'RSH Község', 'RSH Cím/Utca', 'RSH Cím/Szám', 'RSH Cím/Irányítószám', 'RSH Jogi forma',
       'RSH Egység típusa', 'RSH Anyaintézmény', 'RSH Működési forma', 'RSH Finanszírozás',
       'RSH Tulajdonviszony', 'RSH Adószám', 'RSH Megye', 'RSH Módosítva',
       'RSH Akkreditálva', 'RSH Működés kezdete', 'RSH Bezárás ideje',
       'RSH Telefon', 'RSH Fax', 'RSH Email', 'GOOGLE Név', 'GOOGLE Település', 'GOOGLE Község', 'GOOGLE Cím', 
              'GOOGLE koordináta', 'GOOGLE Telefon', 'GOOGLE weboldal', 'GOOGLE térkép', 'GOOGLE fénykép', 
         ]

In [321]:
gata['Név (normalizált)']=gata['Név (normalizált)']\
.str.replace('SGMZ','ÁLTALÁNOS ISKOLA')\
.str.replace('SPRM','ELEMI ISKOLA')\
.str.replace('SPSTL','POSZTLÍCEUM')\
.str.replace('LICTEH','SZAKLÍCEUM')\
.str.replace('LISPRT','SPORTISKOLA')\
.str.replace('CLBCOP','GYEREK-KLUB')\
.str.replace('LITEOR','ELMÉLETI LÍCEUM')\
.str.replace('LIPDGA','TANÍTÓKÉPZŐ')\
.str.replace('LITOLX','TEOLÓGIAI LÍCEUM')\
.str.replace('LIARTE','MŰVÉSZETI LÍCEUM')\
.str.replace('COLGNAT','NEMZETI KOLLÉGIUM')\
.str.replace('GRDNRM','ÓVODA')\
.str.replace('GRDPLG','NAPKÖZI-OTTHON')\
.str.replace('INSPSCJ','TANFELÜGYELŐSÉG')\
.str.replace('SCSPC','SPECIÁLIS ISKOLA')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [323]:
hata=gata.set_index('ID').sort_values(['Év','Megye','Település (eredeti)','Név (normalizált)'])

In [324]:
hata.to_excel('data/output/Erdely_draft_output.xlsx')

DEPRECATED - google cant translate place names
Finish up the rest with `googletrans`

In [22]:
# !pip install googletrans

In [24]:
from googletrans import Translator
translator = Translator()
t=translator.translate('scoala',src='ro',dest='hu')

In [37]:
to_translate=list((data['telepules']+', judetul '+data['Judet']+', ROMANIA').unique())[:20]

In [41]:
to_translate=list((data['Denumire'].astype(str)+' '+data['Localitate'].astype(str)+', '+data['Localitate superioară'].astype(str)+', '+\
     data['Stradă'].astype(str)+' nr. '+data['Număr'].astype(str)+', '+data['Cod poștal'].astype(str).str[:-2]+', '+\
     data['Judet'].astype(str)+', ROMANIA').unique())[:10]

In [42]:
translated={}

In [43]:
translations = translator.translate(to_translate, src='ro', dest='hu')
for translation in translations:
    translated[translation.origin]=translation.text

In [44]:
translated

{'CLUBUL COPIILOR BLAJ BLAJ, MUNICIPIUL BLAJ,  Piata 1848 nr.  8, 515400, AB, ROMANIA': 'BLAJ BLAJ GYERMEKEK KLUBJA, BLAJ KÖZÖSSÉG, Piata 1848 no. 8, 515400, AB, ROMÁNIA',
 'SCOALA GIMNAZIALA "NICOLAE DRAGAN" GALDA DE JOS GALDA DE JOS, GALDA DE JOS, PRINCIPALA nr. 247, 517285, AB, ROMANIA': '"NICOLAE DRAGAN" GYMNASIUM ISKOLA GALDA DE JOS GALDA DE JOS, GALDA DE JOS, PRINCIPALA nr. 247, 517285, AB, ROMÁNIA',
 'SCOALA GIMNAZIALA "AXENTE SEVER" AIUD AIUD, MUNICIPIUL AIUD, UNIRII nr. 9, 515200, AB, ROMANIA': '"AXENTE SEVER" GYMNASIUM ISKOLA AIUD AIUD, MUNICIPIUL AIUD, UNIRII no. 9, 515200, AB, ROMÁNIA',
 'SCOALA GIMNAZIALA "AVRAM IANCU " UNIREA UNIREA, UNIREA, AVRAM IANCU nr. 69, 517785, AB, ROMANIA': '"AVRAM IANCU" GYMNASIUM ISKOLA UNIREA UNIREA, UNIREA, AVRAM IANCU no. 69, 517785, AB, ROMÁNIA',
 'SCOALA GIMNAZIALA "SIMION LAZAR" LUNCA MURESULUI LUNCA MUREŞULUI, LUNCA MUREŞULUI, GARII nr. 241 A, 517405, AB, ROMANIA': '"SIMION LAZAR" GYMNASIUM ISKOLA LUNCA MURESULUI LUNCA MUREŞULUI, LUNCA M